In [1]:
import numpy as np
import numpy.linalg as npl
import numpy.fft as fft
import scipy as sp
import scipy.linalg as spl
import scipy.sparse as sps
import matplotlib.pyplot as plt
#%matplotlib inline
%matplotlib widget

pi = np.pi

# Exercice 1 : Equation de transport

$$u^{n+1}_{j} = u^{n-1}_j + c \frac{\partial t}{\partial x} (u^n_{j+1} - u^n_{j-1})$$

In [2]:
def u0(x, mu=0, sigma2=1):
    exp = np.exp(-0.5 * ((x-mu)**2)/sigma2)
    return exp/np.sqrt(2*pi*sigma2)

In [3]:
def v0(x, k=0.5, mu=0, sigma2=1):
    return u0(x, mu, sigma2) * np.sin(2*k*pi*x)

In [4]:
def w0(x, a=-2.5, b=2.5):
    return np.where((x>=a) & (x<=b), 1, 0)

In [5]:
def sch_Saute_Mouton(U0, U1, c, dt, dx, T):
    U = [U0, U1]
    N = U0.size
    A = c*dt/dx * sps.diags([1, -1, 1, -1], [-N, -1, 1, N], shape=[N, N]).toarray() 
    
    n = 2
    
    while n*dt <= T:
        U2 = U[-2] - A.dot(U[-1])
        U.append(U2)
        n += 1
    
    return U

In [6]:
def sch_décentré_amont(U0, c, dt, dx, T):
    U = [U0]
    N = U0.size
    A = np.eye(N) - c*dt/(dx) * sps.diags([-1, 1, -1], [-1, 0, N-1], shape=[N, N]).toarray()
    
    n = 1
    
    while n*dt <= T:
        U1 = A.dot(U[-1])
        U.append(U1)
        n += 1
        
    return U

In [7]:
def test_sch_décentré_amont(min, max, c, dt=0.005, dx=0.01, T=0.5):
    x = np.linspace(min, max, int((max-min)/dx))
    for func in [u0, v0, w0]:
        U = sch_décentré_amont(func(x), c, dt, dx, T)
        plt.figure(clear=True)
        plt.plot(x ,U[0])
        plt.plot(x, U[-1])
        plt.show()

In [8]:
test_sch_décentré_amont(-5, 5, c=0.5, dt=0.001, dx=0.01, T=4)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
def sch_décentré(U0, c, dt, dx, T):
    U = [U0]
    N = U0.size
    A = np.eye(N) + c*dt/dx * sps.diags([-0.5, 1.5, -1, -0.5, 1.5], [-N+1, -1, 0, 1, N-1], shape=[N, N]).toarray()
    
    n = 1
    
    while n*dt <= T:
        U1 = A.dot(U[-1])
        U.append(U1)
        n += 1
        
    return U

In [10]:
def test_sch_décentré(min, max, c, dt=0.005, dx=0.01, T=0.5):
    x = np.linspace(min, max, int((max-min)/dx))
    for func in [u0, v0, w0]:
        U = sch_décentré(func(x), c, dt, dx, T)
        plt.figure(clear=True)
        plt.plot(x ,U[0])
        plt.plot(x, U[-1])
        plt.show()

In [11]:
test_sch_décentré(-5, 5, c=0.5, dt=0.005, dx=0.01, T=4)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
plt.close('all')

## Exercice 2 : Visualiser les fonctions d'un espace de Sobolev

In [13]:
def sobolev(alpha=1, N=2**9, pha=None):
    K, L = np.meshgrid(np.arange(N), np.arange(N))
    module = (1 + K**2 + L**2)**(-alpha/2)
    
    if pha is None:
        phase = np.random.random((N, N))*2*pi
        coeffs = module * np.exp(1j * phase)
    
    coeffs = module * np.exp(1j * pha)
    f = fft.ifft2(coeffs)
    
    return np.real(f)

In [14]:
fig = plt.figure(figsize=(12, 12))
N = 2**9
seed = np.random.randint(10000)
np.random.seed()
phase = np.random.random((N, N))*2*pi
for i in range(1, 5):
    
    ax = fig.add_subplot(220+i)
    image = sobolev(i, N, phase)
    ax.imshow(image)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib widget

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

phase = np.random.random((N, N))*2*pi
X, Y = np.meshgrid(np.arange(N), np.arange(N))

image = sobolev(2, N, phase)
    
ax.plot_surface(X, Y, image, cmap="Purples")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
images = []

seed = np.random.randint(10000)
np.random.seed()
phase = np.random.random((N, N))*2*pi
for i in np.linspace(1, 5, num=50):
    image = sobolev(i, N, phase)
    images.append(image)

In [44]:
len(images)
plt.figure()
plt.imshow(images[0])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")

N = 2**9

r = np.arange(N)
X, Y = np.meshgrid(r, r)
line = ax.plot_surface(X, Y, images[0], cmap="Purples")


def init():  # only required for blitting to give a clean slate.
    #line.set_ydata([np.nan] * len(x))
    return line,


def animate(i):
    line.set_ydata(images[i]) # update the data.
    #line.set_label(str(i*dt))
    return line,


ani = animation.FuncAnimation(
    fig, animate, init_func=init, interval=5, blit=True)

ani.save('test.gif', writer='imagemagick')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

MovieWriter stderr:
convert: unexpected end-of-file '-': No such file or directory @ error/rgb.c/ReadRGBImage/246.
convert: no images defined `test.gif' @ error/convert.c/ConvertImageCommand/3273.



CalledProcessError: Command '['convert', '-size', '432x288', '-depth', '8', '-delay', '0.5', '-loop', '0', 'rgba:-', 'test.gif']' returned non-zero exit status 1.

In [48]:
# First import everthing you need
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D

N = 2**9

r = np.arange(N)
X, Y = np.meshgrid(r, r)

# Create a figure and a 3D Axes
fig = plt.figure()
ax = Axes3D(fig)

# Create an init function and the animate functions.
# Both are explained in the tutorial. Since we are changing
# the the elevation and azimuth and no objects are really
# changed on the plot we don't have to return anything from
# the init and animate function. (return value is explained
# in the tutorial.
def init():
    ax.plot_surface(X, Y, images[0], cmap="Purples")
    return fig,

def animate(i):
    fig.clf()
    ax.plot_surface(X, Y, images[i], cmap="Purples")
    return fig,

# Animate
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=49, interval=20, blit=True)
# Save
anim.save('basic_animation.mp4', fps=24, extra_args=['-vcodec', 'libx264'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
from ipywidgets import interact, widgets

In [27]:
interact(sobolev, alpha=[1, 10, 0.5])

ValueError: None cannot be transformed to a widget

In [28]:
def f(x):
    return x

In [29]:
interact(f, x=(1,10))

interactive(children=(IntSlider(value=5, description='x', max=10, min=1), Output()), _dom_classes=('widget-int…

<function __main__.f(x)>

In [36]:
def plot_sobolev(alpha=1, seed=100):
    fig = plt.figure()
    ax = fig.add_subplot(111)

    N = 2**7
    np.random.seed(seed)
    phase = np.random.random((N, N))*2*pi

    image = sobolev(alpha, N, phase)
    
    ax.imshow(image)
    plt.show()

In [38]:
interact(plot_sobolev, alpha=(1,5, 0.1))

interactive(children=(FloatSlider(value=1.0, description='alpha', max=5.0, min=1.0), IntSlider(value=100, desc…

<function __main__.plot_sobolev(alpha=1, seed=100)>

In [15]:
plt.close('all')

In [16]:
fig = plt.figure(figsize=(10, 3))

<Figure size 720x216 with 0 Axes>

In [17]:
fig.add_subplot()

## Exercice 3

In [16]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [17]:
def f(x):
    return x

In [18]:
interact(f, x=10)

interactive(children=(IntSlider(value=10, description='x', max=30, min=-10), Output()), _dom_classes=('widget-…

<function __main__.f(x)>

In [19]:
interact(f, x=True);

interactive(children=(Checkbox(value=True, description='x'), Output()), _dom_classes=('widget-interact',))

In [20]:
interact(f, x='Hi there!');

interactive(children=(Text(value='Hi there!', description='x'), Output()), _dom_classes=('widget-interact',))

In [21]:
@interact(x=True, y=1.0)
def g(x, y):
    return (x, y)

interactive(children=(Checkbox(value=True, description='x'), FloatSlider(value=1.0, description='y', max=3.0, …

In [22]:
def h(p, q):
    return (p, q)

In [23]:
interact(h, p=5, q=fixed(20))

interactive(children=(IntSlider(value=5, description='p', max=15, min=-5), Output()), _dom_classes=('widget-in…

<function __main__.h(p, q)>

In [27]:
interact(f, x=widgets.IntSlider(min=-10, max=30, step=1, value=10))

interactive(children=(IntSlider(value=10, description='x', max=30, min=-10), Output()), _dom_classes=('widget-…

<function __main__.f(x)>

In [29]:
interact(f, x=['orange','apple'])

interactive(children=(Dropdown(description='x', options=('orange', 'apple'), value='orange'), Output()), _dom_…

<function __main__.f(x)>

In [31]:
@interact(x=(0.0,20.0,0.5))
def h(x=5.5):
    return x



interactive(children=(FloatSlider(value=5.5, description='x', max=20.0, step=0.5), Output()), _dom_classes=('w…

In [32]:
interact(f, x=[('one', 10), ('two', 20)]);

interactive(children=(Dropdown(description='x', options=(('one', 10), ('two', 20)), value=10), Output()), _dom…

In [33]:
from IPython.display import display
def f(a, b):
    display(a + b)
    return a+b

In [34]:
w = interactive(f, a=10, b=20)

In [35]:
type(w)

ipywidgets.widgets.interaction.interactive

In [36]:
w

interactive(children=(IntSlider(value=10, description='a', max=30, min=-10), IntSlider(value=20, description='…

In [37]:
w.children

(IntSlider(value=10, description='a', max=30, min=-10),
 IntSlider(value=20, description='b', max=60, min=-20),
 Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '30'}, 'metadata': {}},)))

In [38]:
display(w)

interactive(children=(IntSlider(value=10, description='a', max=30, min=-10), IntSlider(value=20, description='…

In [39]:
w.result

30

In [46]:
def slow_function(i):
    print(int(i),list(x for x in range(int(i)) if
                str(x)==str(x)[::-1] and
                str(x**2)==str(x**2)[::-1]))
    return

In [48]:
%%time
slow_function(1e6)

1000000 [0, 1, 2, 3, 11, 22, 101, 111, 121, 202, 212, 1001, 1111, 2002, 10001, 10101, 10201, 11011, 11111, 11211, 20002, 20102, 100001, 101101, 110011, 111111, 200002]
CPU times: user 568 ms, sys: 0 ns, total: 568 ms
Wall time: 566 ms


In [49]:
from ipywidgets import FloatSlider
interact(slow_function,i=FloatSlider(min=1e5, max=1e7, step=1e5))

interactive(children=(FloatSlider(value=100000.0, description='i', max=10000000.0, min=100000.0, step=100000.0…

<function __main__.slow_function(i)>

In [50]:
interact_manual(slow_function,i=FloatSlider(min=1e5, max=1e7, step=1e5))

interactive(children=(FloatSlider(value=100000.0, description='i', max=10000000.0, min=100000.0, step=100000.0…

<function __main__.slow_function(i)>

In [51]:
slow = interactive(slow_function, {'manual': True}, i=widgets.FloatSlider(min=1e4, max=1e6, step=1e4))
slow

interactive(children=(FloatSlider(value=10000.0, description='i', max=1000000.0, min=10000.0, step=10000.0), B…

In [52]:
interact(slow_function,i=FloatSlider(min=1e5, max=1e7, step=1e5, continuous_update=False))

interactive(children=(FloatSlider(value=100000.0, continuous_update=False, description='i', max=10000000.0, mi…

<function __main__.slow_function(i)>

In [54]:
a = widgets.IntSlider()
b = widgets.IntSlider()
c = widgets.IntSlider()
ui1 = widgets.HBox([a, b])
ui2 = widgets.HBox([c])
ui = widgets.VBox([ui1, ui2])
def f(a, b, c):
    print((a, b, c))

out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

display(ui, out)

Output()

In [56]:
x_widget = FloatSlider(min=0.0, max=10.0, step=0.05)
y_widget = FloatSlider(min=0.5, max=10.0, step=0.05, value=5.0)

def update_x_range(*args):
    x_widget.max = 2.0 * y_widget.value
y_widget.observe(update_x_range, 'value')

@interact(x=x_widget, y=y_widget)
def printer(x, y):
    print(x, y)

interactive(children=(FloatSlider(value=0.0, description='x', max=10.0, step=0.05), FloatSlider(value=5.0, des…

In [58]:
%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

def f(m, b):
    plt.figure(2)
    x = np.linspace(-10, 10, num=1000)
    plt.plot(x, m * x**3 + b*x)
    plt.ylim(-5, 5)
    plt.show()

interactive_plot = interactive(f, m=(-2.0, 2.0), b=(-3, 3, 0.5))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=0.0, description='m', max=2.0, min=-2.0), FloatSlider(value=0.0, descr…